In [72]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
import data_preprocessing
import importlib
importlib.reload(data_preprocessing)
import cross_validation
import TEST
importlib.reload(TEST)
importlib.reload(cross_validation)



<module 'cross_validation' from '/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/cross_validation.py'>

In [3]:
np.random.seed(42)

In [4]:
#Load Data
x_train_raw, x_test_raw,y_train_raw, train_ids, test_ids = load_csv_data('dataset')

In [1]:
print(x_train_raw.shape)
print(x_test_raw.shape)


NameError: name 'x_train_raw' is not defined

In [43]:
#Reformat y_train to have 0's instead of -1's
y_train_original = y_train_raw.copy()
y_train_raw[y_train_raw == -1] = 0

In [44]:
#replace NaN by a float to treat NaN as a categorical feature
x_test_raw = np.nan_to_num(x_test_raw, nan = -10.0)
x_test_raw.shape

(109379, 321)

### One Hot Encode

In [7]:
data_annoted = data_preprocessing.read_annotated_csv('dataset/data_anotated.csv')

In [73]:
x_train, y_train, x_test = data_preprocessing.preprocess_data2(x_train_raw[:50000,:], y_train_raw[:50000], x_test_raw[:50000,:], data_annoted)


/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/data_preprocessing.py:134: RuntimeWarning: invalid value encountered in divide
  data = data / data.max(axis=0)


In [74]:
x_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [47]:
"""x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)
x_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)"""

'x_train_filtered = data_preprocessing.remove_useless(x_train_raw, data_annoted)\nx_test_filtered = data_preprocessing.remove_useless(x_test_raw, data_annoted)'

In [48]:
"""data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)
data_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)"""

'data_train = data_preprocessing.clean_data(x_train_filtered[:100000,:], data_annoted)\ndata_test = data_preprocessing.clean_data(x_test_filtered[:100000,:], data_annoted)'

In [75]:
np.savetxt('x_train_preprocessed.csv', x_train, delimiter=',')
np.savetxt('y_train_preprocessed.csv', y_train, delimiter=',')
np.savetxt('x_test_preprocessed.csv', x_test, delimiter=',')

In [76]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')

In [ ]:
import implemented_functions
import importlib
importlib.reload(implemented_functions)

initial_w = np.zeros(x_train.shape[1])
max_iters = 200

x_test = x_train[7000:,:]
y_test = y_train[7000:]
x_train = x_train[:7000,:]
y_train = y_train[:7000]


w_least_squares, loss_ls = implemented_functions.least_squares(y_train, x_train)
print(f"Least Squares: Loss = {loss_ls}")

best_mse_coef = 0
best_mse_loss = np.inf
best_mse_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_sgd, loss_sgd = implemented_functions.mean_squared_error_sgd(y_train,  x_train, initial_w, max_iters, gamma)
    if loss_sgd[-1] < best_mse_loss:
        best_mse_loss = loss_sgd[-1]
        best_mse_coef = gamma
        best_mse_weights = w_sgd
    print(f"Mean Squared Error SGD (gamma={gamma}): Loss = {loss_sgd}")

best_log_lasso_coef = 0
best_log_lasso_loss = np.inf
best_log_lasso_weights = None
for gamma in [0.01, 0.1, 0.5]:
    for lambda_ in [0.1, 0.01]:
        w_log_lasso, loss_log_lasso = implemented_functions.reg_logistic_lasso_subgradient(y_train, x_train, lambda_, initial_w, max_iters, gamma)
        if loss_log_lasso < best_log_lasso_loss:
            best_log_lasso_loss = loss_log_lasso
            best_log_lasso_weights = w_log_lasso
        print(f"Regularized Lasso Logistic Regression (gamma={gamma}, lambda={lambda_}): Loss = {loss_log_lasso}")
        
best_ridge_coef = 0
best_ridge_loss = np.inf
best_ridge_weights = None
for ridge_coef in [0.001, 0.01, 0.1, 1, 10]:
    w_ridge, loss_ridge = implemented_functions.ridge_regression(y_train, x_train, ridge_coef)
    if loss_ridge < best_ridge_loss:
        best_ridge_loss = loss_ridge
        best_ridge_coef = ridge_coef
        best_ridge_weights = w_ridge
    print(f"Ridge Regression (alpha={ridge_coef}): Loss = {loss_ridge}")

best_log_gamma = 0
best_log_loss = np.inf
best_log_weights = None
for gamma in [0.01, 0.1, 0.5, 1]:
    w_logistic, loss_logistic = implemented_functions.logistic_regression(y_train, x_train, initial_w, max_iters, gamma)
    if loss_logistic < best_log_loss:
        best_log_loss = loss_logistic
        best_log_gamma = gamma
        best_log_weights = w_logistic
    print(f"Logistic Regression: Loss = {loss_logistic}")

best_log_reg_coef = 0
best_log_reg_loss = np.inf
best_log_reg_weights = None
for gamma in [0.01, 0.1, 0.5]:
    for reg_coef in [0.001, 0.01, 0.1, 1]:
        w_reg_logistic, loss_reg_logistic = implemented_functions.reg_logistic_regression(y_train, x_train, reg_coef, initial_w, max_iters, gamma)
        if loss_reg_logistic < best_log_reg_loss:
            best_log_reg_loss = loss_reg_logistic
            best_log_reg_coef = reg_coef
            best_log_reg_weights = w_reg_logistic
        print(f"Regularized Logistic Regression (alpha={reg_coef}, gamma={gamma}): Loss = {loss_reg_logistic}")



/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:140: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(tx,y)[0]


Least Squares: Loss = 0.07250842162719422
Mean Squared Error SGD (gamma=0.01): Loss = [687.474503065241, 224.87298065310233, 51.09998383944982, 75.51102121097257, 76.57240565137145, 34.35787434476492, 28.004505839975298, 27.319480632881668, 28.644884751802365, 36.819451711367336, 28.584977617779952, 54.45023228775416, 53.46253633298848, 42.607207352745775, 31.084146737880335, 27.450013222063458, 34.53096182189085, 28.148066155293396, 30.721367700023468, 25.886012562409878, 26.725426053219074, 25.304105235427677, 26.090743026306654, 48.69306824429197, 20.278735774094088, 20.298550193802807, 20.97998668234299, 21.893182934592325, 26.76774277407371, 20.181383900542098, 20.2955556499679, 19.504398263862225, 27.845582997138642, 18.10977878155092, 18.37853613054735, 21.291927688522875, 22.007624633806426, 18.61795464701691, 20.848223899290744, 20.512852774116705, 18.356017057729343, 17.436216151668315, 17.291270557743683, 20.96355489231087, 18.4025556328858, 17.347879454234445, 21.3614709823

/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:53: RuntimeWarning: overflow encountered in matmul
  MSE_loss = 1 / (2*N) * (e.T @ e)


Mean Squared Error SGD (gamma=0.5): Loss = [1270700.6906089094, 279092462.7689524, 155530766740.87595, 151391601155299.1, 7.276751944791406e+16, 4.396765432652231e+19, 2.004136819913084e+22, 1.1098928662979652e+25, 1.9488640334331615e+27, 3.500192061572821e+29, 8.830123624988986e+31, 2.1048593495269464e+34, 3.878749468337094e+36, 1.5916680834760115e+39, 8.01142718046467e+41, 3.666116800694119e+44, 1.1230433761780402e+47, 4.5112189461855983e+49, 2.858009958215713e+52, 1.1399018176195642e+55, 4.487726550292878e+57, 2.8013489021952072e+60, 6.467293134894199e+62, 3.662647494756325e+65, 1.7908452296771525e+68, 5.758214600451294e+70, 3.6638360696041225e+73, 1.637801294074021e+76, 4.345322218202996e+78, 1.1593308575445217e+81, 5.454600202808853e+83, 2.0983893101548602e+86, 1.2378940361147226e+89, 5.122557762647681e+91, 2.740240307078285e+94, 2.0964355523376766e+97, 6.644272894756964e+99, 3.1819992459028734e+102, 3.6835395937138015e+104, 1.2763650602626116e+107, 7.006383772614187e+109, 2.04816

/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:52: RuntimeWarning: overflow encountered in matmul
  e = y - tx @ w
/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:74: RuntimeWarning: overflow encountered in matmul
  e = y- tx@w
/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:75: RuntimeWarning: invalid value encountered in matmul
  return -(tx.T@e)/n
/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:52: RuntimeWarning: invalid value encountered in matmul
  e = y - tx @ w
/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Le

Regularized Lasso Logistic Regression (gamma=0.01, lambda=0.1): Loss = 48.80091900535621
Regularized Lasso Logistic Regression (gamma=0.01, lambda=0.01): Loss = 20.25490559803804
Regularized Lasso Logistic Regression (gamma=0.1, lambda=0.1): Loss = 0.9287527521035257
Regularized Lasso Logistic Regression (gamma=0.1, lambda=0.01): Loss = 4.346407334254343
Regularized Lasso Logistic Regression (gamma=0.5, lambda=0.1): Loss = 5.1831435696564405
Regularized Lasso Logistic Regression (gamma=0.5, lambda=0.01): Loss = 1.2612558773420381
Ridge Regression (alpha=0.001): Loss = 0.07434071058951153


/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:153: RuntimeWarning: invalid value encountered in matmul
  A = tx.T @ tx + (N * lambda_) * np.eye(D)


Ridge Regression (alpha=0.01): Loss = 0.07687666771243132
Ridge Regression (alpha=0.1): Loss = 0.08179840389956802
Ridge Regression (alpha=1): Loss = 0.09745806649800418
Ridge Regression (alpha=10): Loss = 0.14062682022968842
Logistic Regression: Loss = 15.851271978435785
Logistic Regression: Loss = 1.1988288819514517
Logistic Regression: Loss = 0.5986628835079466
Logistic Regression: Loss = 1.541105683237969
Regularized Logistic Regression (alpha=0.001, gamma=0.01): Loss = 16.05150726205191
Regularized Logistic Regression (alpha=0.01, gamma=0.01): Loss = 17.754772171730995
Regularized Logistic Regression (alpha=0.1, gamma=0.01): Loss = 27.32840376358681
Regularized Logistic Regression (alpha=1, gamma=0.01): Loss = 4.50476953636982
Regularized Logistic Regression (alpha=0.001, gamma=0.1): Loss = 1.3666039368893967
Regularized Logistic Regression (alpha=0.01, gamma=0.1): Loss = 2.315264121337969
Regularized Logistic Regression (alpha=0.1, gamma=0.1): Loss = 0.8639985871139272
Regularize

In [78]:
def compute_accuracy(y_test, x_test, w, method):
    y_pred = x_test@w
    y_pred[y_pred <= 0.5] = 0
    y_pred[y_pred > 0.5] = 1
    computed_accuracy = np.sum(y_pred == y_test)/len(y_test)
    print(f"Accuracy of {method} is {computed_accuracy*100}%")
    return computed_accuracy, y_pred

def test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights):
    accuracy, y_pred = compute_accuracy(y_test, x_test, w_sgd, "SGD")
    print(f"F1 score SGD is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = compute_accuracy(y_test, x_test, w_least_squares, "Least Squares")
    print(f"F1 score Least Squares is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = compute_accuracy(y_test, x_test, best_ridge_weights, "Ridge")
    print(f"F1 score Ridge is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = compute_accuracy(y_test, x_test, best_log_weights, "Logistic")
    print(f"F1 score Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    accuracy, y_pred = compute_accuracy(y_test, x_test, best_log_reg_weights, "Regularized Logistic")
    print(f"F1 score Regularized Logistic is {implemented_functions.compute_f1_score(y_test, y_pred)}")
    return y_pred

test_methods(x_test, y_test, w_sgd, w_least_squares, best_ridge_weights, best_log_weights, best_log_reg_weights)


Accuracy of SGD is 0.0%
F1 score SGD is nan
Accuracy of Least Squares is 76.47768395657418%
F1 score Least Squares is 0.7616136919315404
Accuracy of Ridge is 77.3220747889023%
F1 score Ridge is 0.7761904761904762
Accuracy of Logistic is 73.8238841978287%
F1 score Logistic is 0.7027397260273973
Accuracy of Regularized Logistic is 73.8238841978287%
F1 score Regularized Logistic is 0.7027397260273973


/Users/christophersoriano/Library/Mobile Documents/com~apple~CloudDocs/Mac/Epfl/Master/MA3/Machine Learning/ML_project1/project1/implemented_functions.py:289: RuntimeWarning: invalid value encountered in scalar divide


array([0., 0., 0., ..., 0., 0., 1.])

In [44]:

X_max = data_preprocessing.build_poly(x_train, 3)
initial_w = np.zeros(X_max.shape[1])

cross_validation.cross_validation_demo(y=y_train, x=x_train, k_fold=5, lambdas=[0.001], degree=3, initial_w=initial_w, max_iters=100, gamma=0.1)

KeyboardInterrupt: 

In [19]:
x_train = np.loadtxt('x_train_preprocessed.csv', delimiter=',')
x_test = np.loadtxt('x_test_preprocessed.csv', delimiter=',')
y_train = np.loadtxt('y_train_preprocessed.csv', delimiter=',')
y_pred = x_test@best_ridge_weights


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 575 is different from 561)

In [16]:
best_ridge_weights.shape

(575,)

In [20]:
x_train.shape

(8658, 575)

In [22]:
x_test.shape

(50000, 561)